In [5]:
import sys, os
import numpy as np
import open3d as o3d
import os.path as osp
import matplotlib.pyplot as plt
from transforms.transforms import Identity
from datasets.kitti_car_instances import KittiCarInstances
from models import PointConvSceneFlowPWC8192selfglobalPointConv as PointConvSceneFlow

import torch.utils.data as data

data_process_args = {'DEPTH_THRESHOLD':0, 'NO_CORR':True}
identity_transform = Identity(data_process_args=data_process_args, num_points=8192)
dataset = KittiCarInstances(train=True, data_root='/external/datasets/two_frame_car/pcds/train', 
                            transform=identity_transform, num_points=8192, remove_ground=False)

# pcd0 = o3d.geometry.PointCloud()
# pcd0.points = o3d.utility.Vector3dVector(dataset[5][0])
# o3d.visualization.draw_geometries([pcd0])

# pcd1 = o3d.geometry.PointCloud()
# pcd1.points = o3d.utility.Vector3dVector(dataset[5][2])
# o3d.visualization.draw_geometries([pcd1])

In [11]:
import torch
model = PointConvSceneFlow().cuda()
model.load_state_dict(torch.load('ckpts/PointConv_639_1.1546.pth'))

model.train = False

pc1, pc2, norm1, norm2, _, _ = dataset[0]
pc1 = torch.as_tensor(pc1).unsqueeze(0).cuda().float()
pc2 = torch.as_tensor(pc2).unsqueeze(0).cuda().float()
norm1 = torch.as_tensor(norm1).unsqueeze(0).cuda().float()
norm2 = torch.as_tensor(norm2).unsqueeze(0).cuda().float()

pred_flows, _, _, pc1, pc2 = model(pc1, pc2, norm1, norm2)

In [12]:
pts0 = pc1[0].cpu().numpy().squeeze().T
flow0 = pred_flows[0].detach().cpu().numpy().squeeze().T
pts0_warp = pts0 + flow0
pcd0 = o3d.geometry.PointCloud()
pcd0.points = o3d.utility.Vector3dVector(pts0)
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(pts0_warp)

o3d.io.write_point_cloud('out_pcds/pts0.pcd', pcd0, print_progress=True)
o3d.io.write_point_cloud('out_pcds/pts0_warp.pcd', pcd1, print_progress=True)

# o3d.visualization.draw_geometries([pcd0, pcd1])
# o3d.visualization.draw_geometries([pcd0])
# o3d.visualization.draw_geometries([pcd1])

True